# Projeto 1 - Ciência dos Dados

Nome: Luiz Ricardo Hardman Paranhos

Nome: Matheus Kwon

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [43]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [44]:
import os

filename = 'cheetos.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo cheetos.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [45]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,relevancia
0,@eaipaixao_ cheetos obv,1.0
1,@giovannapinto mano eu fui comprar 2 cheetos e...,1.0
2,@zabele__ só tenho cheetos,0.0
3,"@lilizvante cheetos, old\nhttps://t.co/atamf2woj7",0.0
4,@biaventura14 kkkkkkkkkkkkkkkkkkkkk cheetos ve...,1.0


In [46]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,relevancia
0,sou do tipo que quando tá conversando com algu...,0
1,att fresquinha de flaming hot cheetos!!!! http...,1
2,me entupindo de cheetos enquanto vejo tapas e ...,1
3,cheetos ou doritos https://t.co/caytqjylnp,1
4,mas quem colocou o cheetos na minha tml hoje ?,1


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

- Relevante: informação da qual a empresa pode ganhar dinheiro
- Irrelevante: resto

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [47]:
train['relevancia'] = train['relevancia'].astype('category')
train.relevancia.cat.categories = ['irrelevante', 'relevante']

In [48]:
# https://docs.python.org/3/library/re.html#
import re 


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;/,_@¿]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

def cleanup_enter(text):
    punctuation = '\n'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed

def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                            "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

def link(text):
    text = re.sub(r'http\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'@\S+', '', text, flags=re.MULTILINE)
    return text

In [49]:
# String com todas as palavras
tweets_treinamento_raw = ''
for tweet in train.Treinamento:
    tweets_treinamento_raw = tweets_treinamento_raw + ' ' + tweet
tweets_treinamento = deEmojify(cleanup_enter(cleanup(link(tweets_treinamento_raw.lower()))))

In [50]:
# String com os relevantes
filtro_relevante_train = train.relevancia == 'relevante'
treinamento_relevante = train.loc[filtro_relevante_train,:]

tweets_relevantes = ''
for tweet in treinamento_relevante.Treinamento:
    tweets_relevantes = tweets_relevantes + ' ' + tweet

treinamento_relevante = deEmojify(cleanup_enter(cleanup(link(tweets_relevantes.lower()))))

In [51]:
# String com os irrelevantes
filtro_irrelevante_train = train.relevancia == 'irrelevante'
treinamento_irrelevante = train.loc[filtro_irrelevante_train,:]

tweets_irrelevantes = ''
for tweet in treinamento_irrelevante.Treinamento:
    tweets_irrelevantes = tweets_irrelevantes + ' ' + tweet

    
treinamento_irrelevante = deEmojify(cleanup_enter(cleanup(tweets_irrelevantes.lower())))

In [52]:
# Criando lista e séries a partir das strings
tweets_treinamento_lista = tweets_treinamento.split()
treinamento_relevante_lista = treinamento_relevante.split()
treinamento_irrelevante_lista = treinamento_irrelevante.split()

serie_treinamento = pd.Series(tweets_treinamento_lista)
serie_treinamento_relevante = pd.Series(treinamento_relevante_lista)
serie_treinamento_irrelevante = pd.Series(treinamento_irrelevante_lista)

In [53]:
tabela_treinamento = serie_treinamento.value_counts()
tabela_treinamento

cheetos           297
de                171
e                 116
o                  81
um                 81
                 ... 
relacionamento      1
fossa               1
matei               1
ryujinijkkk         1
exatamente          1
Length: 1236, dtype: int64

In [54]:
tabela_treinamento_relevante = serie_treinamento_relevante.value_counts()
tabela_treinamento_relevante

cheetos       151
de             92
e              48
um             39
eu             31
             ... 
depressor       1
acordando       1
merendava       1
degustei        1
exatamente      1
Length: 554, dtype: int64

In [55]:
stopwords = ['por', 'se','uma','em','mais','q','que','na','da','do','no','eu','de','e','o','um','a','com','pra', 'mas', 'só', 'so']
def remove_stopwords(lista):
    nova_lista = []
    for palavra in lista:
        x = 0
        if palavra in stopwords:
            x = 1
        if x == 0:
            if palavra[-3:] == 'ndo' or palavra[-3:] == 'mos':
                if palavra == 'quando' or palavra == 'mando':
                    nova_lista.append(palavra)
                else:
                    nova_lista.append(palavra[:-3]+'r')
            elif palavra == 'n' or palavra == 'nao':
                nova_lista.append('não')
            elif palavra == 'ta' or palavra =='tá':
                nova_lista.append('está')
            else:
                nova_lista.append(palavra)
    return nova_lista

In [56]:
tweets_treinamento_stopwords_lista = remove_stopwords(tweets_treinamento_lista)
        

serie_treinamento_filtrado = pd.Series(tweets_treinamento_stopwords_lista)
tabela_treinamento_filtrado = serie_treinamento_filtrado.value_counts()
tabela_treinamento_filtrado_relativa = serie_treinamento_filtrado.value_counts(True)
tabela_treinamento_filtrado.head(20)

cheetos      297
é             60
não           50
comer         38
requeijão     31
doritos       26
tem           26
meu           23
me            19
gosto         14
melhor        13
pq            13
vou           12
minha         12
cheiro        12
vontade       12
queijo        11
está          11
como          11
fandangos     10
dtype: int64

In [57]:
# Removendo stop words da lista de todas as palavras relevantes
treinamento_relevante_stopwords_lista = remove_stopwords(treinamento_relevante_lista)

serie_treinamento_relevante_filtrado = pd.Series(treinamento_relevante_stopwords_lista)
tabela_treinamento_relevante_filtrado = serie_treinamento_relevante_filtrado.value_counts()
tabela_treinamento_relevante_filtrado_relativa = serie_treinamento_relevante_filtrado.value_counts(True)
tabela_treinamento_relevante_filtrado.head(20)
# palavras_relevantes_totais = len(tabela_treinamento_relevante_filtrado)
# primeiras_relevantes = tabela_treinamento_relevante_filtrado.head(20)
# P_Palavras = {}
# for palavras, valor in primeiras_relevantes.items():
#     P_Palavras[palavra] = (valor + 1) / (palavras_totais + palavras_relevantes_totais)
# print(P_Palavras)

cheetos       151
é              30
requeijão      28
comer          24
doritos        19
gosto          12
melhor         11
vontade        11
meu            11
não            10
queria         10
tem             9
me              8
fandangos       8
ou              7
sabor           7
salgadinho      7
pq              7
pacote          7
queijo          6
dtype: int64

In [58]:
# Removendo stop words da lista de todas as palavras irrelevantes
treinamento_irrelevante_stopwords_lista = remove_stopwords(treinamento_irrelevante_lista)

serie_treinamento_irrelevante_filtrado = pd.Series(treinamento_irrelevante_stopwords_lista)
tabela_treinamento_irrelevante_filtrado = serie_treinamento_irrelevante_filtrado.value_counts()
tabela_treinamento_irrelevante_filtrado_relativa = serie_treinamento_irrelevante_filtrado.value_counts(True)
tabela_treinamento_irrelevante_filtrado.head(20)

cheetos    145
não         40
é           30
tem         17
comer       14
meu         12
me          11
vou         10
cheiro      10
ele          9
minha        8
está         8
como         8
seu          7
tipo         7
aqui         7
pq           6
as           6
fazer        6
doritos      6
dtype: int64

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [59]:
P_R = len(treinamento_relevante_stopwords_lista) / len(tweets_treinamento_stopwords_lista)
P_Rc = 1 - P_R

classificador = []
def testar (tweet):
    P_tweetDadoR = 1
    P_tweetDadoRc = 1
    for palavra in tweet_teste:
        if palavra in tabela_treinamento_relevante_filtrado:
            P_tweetDadoR = P_tweetDadoR*(tabela_treinamento_relevante_filtrado[palavra]+1) / (len(tabela_treinamento_filtrado)+len(tabela_treinamento_relevante_filtrado))
            if palavra in tabela_treinamento_irrelevante_filtrado:
                P_tweetDadoRc = P_tweetDadoRc*(tabela_treinamento_irrelevante_filtrado[palavra]+1) / (len(tabela_treinamento_filtrado)+len(tabela_treinamento_irrelevante_filtrado))
            else:
                P_tweetDadoRc = P_tweetDadoRc / (len(tabela_treinamento_filtrado)+len(tabela_treinamento_irrelevante_filtrado))
        else:
            P_tweetDadoR = P_tweetDadoR / (len(tabela_treinamento_filtrado)+len(tabela_treinamento_relevante_filtrado))
            if palavra in tabela_treinamento_irrelevante_filtrado:
                P_tweetDadoRc = P_tweetDadoRc*(tabela_treinamento_irrelevante_filtrado[palavra]+1) / (len(tabela_treinamento_filtrado)+len(tabela_treinamento_irrelevante_filtrado))
            else:
                P_tweetDadoRc = P_tweetDadoRc / (len(tabela_treinamento_filtrado)+len(tabela_treinamento_irrelevante_filtrado))
    P_RDadoTweet = P_tweetDadoR * P_R
    P_RcDadoTweet = P_tweetDadoRc * P_Rc
    if P_RDadoTweet > P_RcDadoTweet:
        classificador.append('relevante')
    else:
        classificador.append('irrelevante')

for tweet in test.Teste:
    tweet_teste = remove_stopwords(deEmojify(cleanup_enter(cleanup(link(tweet.lower())))).split())
    testar(tweet_teste)
test['classificador'] = classificador
test
    
    
    
    
    

,Teste,relevancia,classificador
0,sou do tipo que quando tá conversando com algu...,0,relevante
1,att fresquinha de flaming hot cheetos!!!! http...,1,irrelevante
2,me entupindo de cheetos enquanto vejo tapas e ...,1,relevante
3,cheetos ou doritos https://t.co/caytqjylnp,1,relevante
4,mas quem colocou o cheetos na minha tml hoje ?,1,irrelevante
...,...,...,...
195,"@bebediaboo eu muito li ""anti-cheetos""",0,relevante
196,"salgadinho, cheetos, biscoito recheado, biscoi...",0,relevante
197,queria experimentar takis e hot cheetos,1,relevante
198,@lwttersondre cheetos de requeijão &gt;&gt;&gt...,1,relevante


In [60]:
sentimento = pd.crosstab(test.relevancia, test.classificador, normalize='index')
sentimento

classificador,irrelevante,relevante
relevancia,,
0,0.383838,0.616162
1,0.128713,0.871287


___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**